In [ ]:
!pip install playwright nest_asyncio
!playwright install chromium

import nest_asyncio
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

nest_asyncio.apply()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 9.2 MB/s eta 0:00:00
164.9 MiB [] 0% 0.0s164.9 MiB [] 0% 11.8s164.9 MiB [] 0% 9.1s164.9 MiB [] 0% 7.2s164.9 MiB [] 1% 5.7s164.9 MiB [] 1% 5.3s164.9 MiB [] 2% 5.0s164.9 MiB [] 2% 4.9s164.9 MiB [] 3% 4.8s164.9 MiB [] 3% 4.2s164.9 MiB [] 4% 3.8s164.9 MiB [] 5% 3.4s164.9 MiB [] 6% 3.6s164.9 MiB [] 6% 3.7s164.9 MiB [] 6% 4.0s164.9 MiB [] 6% 4.3s164.9 MiB [] 7% 4.1s164.9 MiB [] 8% 4.0s164.9 MiB [] 8% 3.9s164.9 MiB [] 9% 3.8s164.9 MiB [] 9% 3.9s164.9 MiB [] 10% 3.8s164.9 MiB [] 11% 3.9s164.9 MiB [] 11% 3.8s164.9 MiB [] 12% 3.8s164.9 MiB [] 13% 3.8s164.9 MiB [] 13% 3.7s164.9 MiB [] 14% 3.6s164.9 MiB [] 14% 3.5s164.9 MiB [] 15% 3.4s164.9 MiB [] 16% 3.4s164.9 MiB [] 17% 3.3s164.9 MiB [] 18% 3.3s164.9 MiB [] 18% 3.4s164.9 MiB [] 19% 3.3s164.9 MiB [] 20% 3.3s164.9 MiB [] 20% 3.2s164.9 MiB [] 21% 3.2s164.9 MiB [] 22% 3.2s164.9 MiB [] 22% 3.3s164.9 MiB [] 23% 3.2s164.9 MiB [] 24% 3.2s164.9 MiB [] 25% 3.1s164.9 MiB [] 25% 3.0s164.9 MiB [] 26% 2

## 스킨/토너

In [ ]:
# HTML 요청
async def crawl_skintoner_list_html(playwright_page, page):
    url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010013&pageIdx={page}&rowsPerPage=24"
    await playwright_page.goto(url)
    await playwright_page.wait_for_timeout(3000)
    html = await playwright_page.content()
    return html

# HTML 파싱
def parse_product_list(html, category_name, page_number):
    soup = BeautifulSoup(html, 'lxml')
    items = soup.select(".prd_info")
    data = []

    for idx, item in enumerate(items, start=1):
        try:
            # 제품명
            name_tag = item.select_one(".tx_name")
            product_name = name_tag.get_text(strip=True)

            # 링크 (절대경로 이미 포함 → 추가 연결하지 않음)
            link = item.select_one("a")["href"]
            full_link = link if link.startswith("http") else "https://www.oliveyoung.co.kr" + link

            # 정가 추출 (없으면 현재가와 동일한 것으로 처리)
            price_tag = item.select_one(".tx_org .tx_num")
            if not price_tag:
                price_tag = item.select_one(".tx_cur .tx_num")
            price = price_tag.get_text(strip=True) if price_tag else None

            data.append({
                "category": category_name,
                "page": page_number,
                "index": idx,
                "product_name": product_name,
                "link": full_link,
                "price": price
            })
        except Exception as e:
            print(f"⚠️ {idx}번 항목 파싱 실패 (page {page_number}): {e}")
            continue
    return data

# 크롤링 실행
async def main():
    category_name = "스킨/토너"
    num_pages = 16
    total_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for i in range(1, num_pages + 1):
            print(f"📦 크롤링 중: {i}페이지")
            html = await crawl_skintoner_list_html(page, i)
            data = parse_product_list(html, category_name, i)
            total_data.extend(data)

        await browser.close()

    # CSV 저장
    global df, csv_path
    df = pd.DataFrame(total_data)
    csv_path = "제품명-스킨_토너.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')

    return df

# 실행
df = await main()
df.head()


📦 크롤링 중: 1페이지
📦 크롤링 중: 2페이지
📦 크롤링 중: 3페이지
📦 크롤링 중: 4페이지
📦 크롤링 중: 5페이지
📦 크롤링 중: 6페이지
📦 크롤링 중: 7페이지
📦 크롤링 중: 8페이지
📦 크롤링 중: 9페이지
📦 크롤링 중: 10페이지
📦 크롤링 중: 11페이지
📦 크롤링 중: 12페이지
📦 크롤링 중: 13페이지
📦 크롤링 중: 14페이지
📦 크롤링 중: 15페이지
📦 크롤링 중: 16페이지


,category,page,index,product_name,link,price
0,스킨/토너,1,1,바이오더마 하이드라비오 토너 500ml,https://www.oliveyoung.co.kr/store/goods/getGo...,"38,000"
1,스킨/토너,1,2,[3월올영픽/단독기획] 토리든 다이브인 저분자 히알루론산 토너 500ml 기획 (+...,https://www.oliveyoung.co.kr/store/goods/getGo...,"29,000"
2,스킨/토너,1,3,[1등 속보습] 라네즈 크림스킨 170ml 리필 기획 (+170ml 리필+미스트펌프...,https://www.oliveyoung.co.kr/store/goods/getGo...,"33,000"
3,스킨/토너,1,4,[2024 어워즈] 아누아 어성초 수딩 토너 350ml +350ml 어워즈 한정기획...,https://www.oliveyoung.co.kr/store/goods/getGo...,"49,000"
4,스킨/토너,1,5,[3월 올영픽] 라로슈포제 시카플라스트 로션 B5 판테놀 시카에센스 토너 200ml,https://www.oliveyoung.co.kr/store/goods/getGo...,"36,000"


In [ ]:
from google.colab import files

# 저장된 파일 다운로드
files.download("제품명-스킨_토너.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 에센스/세럼/앰플

In [ ]:
# 최초 실행 시 설치
!pip install -q playwright nest_asyncio
!playwright install chromium

import nest_asyncio
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

nest_asyncio.apply()

# HTML 요청
async def crawl_essence_list_html(playwright_page, page):
    url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010014&pageIdx={page}&rowsPerPage=24"
    await playwright_page.goto(url)
    await playwright_page.wait_for_timeout(3000)
    html = await playwright_page.content()
    return html

# HTML 파싱
def parse_product_list(html, category_name, page_number):
    soup = BeautifulSoup(html, 'lxml')
    items = soup.select(".prd_info")
    data = []

    for idx, item in enumerate(items, start=1):
        try:
            # 제품명
            name_tag = item.select_one(".tx_name")
            product_name = name_tag.get_text(strip=True)

            # 링크 (이미 절대경로면 그대로 사용)
            link = item.select_one("a")["href"]
            full_link = link if link.startswith("http") else "https://www.oliveyoung.co.kr" + link

            # 정가 (없으면 None 처리)
            price_tag = item.select_one(".tx_org .tx_num")
            if not price_tag:
                price_tag = item.select_one(".tx_cur .tx_num")
            price = price_tag.get_text(strip=True) if price_tag else None

            data.append({
                "category": category_name,
                "page": page_number,
                "index": idx,
                "product_name": product_name,
                "link": full_link,
                "price": price
            })
        except Exception as e:
            print(f"⚠️ {idx}번 항목 파싱 실패 (page {page_number}): {e}")
            continue
    return data

# 크롤링 실행
async def main():
    category_name = "에센스/세럼/앰플"
    num_pages = 33
    total_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for i in range(1, num_pages + 1):
            print(f"📦 크롤링 중: {i}페이지")
            html = await crawl_essence_list_html(page, i)
            data = parse_product_list(html, category_name, i)
            total_data.extend(data)

        await browser.close()

    # 저장
    global df, csv_path
    df = pd.DataFrame(total_data)
    csv_path = "제품명-에센스_세럼_앰플.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')

    return df

# 실행
df = await main()
df.head()


📦 크롤링 중: 1페이지
📦 크롤링 중: 2페이지
📦 크롤링 중: 3페이지
📦 크롤링 중: 4페이지
📦 크롤링 중: 5페이지
📦 크롤링 중: 6페이지
📦 크롤링 중: 7페이지
📦 크롤링 중: 8페이지
📦 크롤링 중: 9페이지
📦 크롤링 중: 10페이지
📦 크롤링 중: 11페이지
📦 크롤링 중: 12페이지
📦 크롤링 중: 13페이지
📦 크롤링 중: 14페이지
📦 크롤링 중: 15페이지
📦 크롤링 중: 16페이지
📦 크롤링 중: 17페이지
📦 크롤링 중: 18페이지
📦 크롤링 중: 19페이지
📦 크롤링 중: 20페이지
📦 크롤링 중: 21페이지
📦 크롤링 중: 22페이지
📦 크롤링 중: 23페이지
📦 크롤링 중: 24페이지
📦 크롤링 중: 25페이지
📦 크롤링 중: 26페이지
📦 크롤링 중: 27페이지
📦 크롤링 중: 28페이지
📦 크롤링 중: 29페이지
📦 크롤링 중: 30페이지
📦 크롤링 중: 31페이지
📦 크롤링 중: 32페이지
📦 크롤링 중: 33페이지


,category,page,index,product_name,link,price
0,에센스/세럼/앰플,1,1,[3월 올영픽/파우치 증정] 메디힐 마데카소사이드 흔적 리페어 세럼 40+40mL ...,https://www.oliveyoung.co.kr/store/goods/getGo...,"36,900"
1,에센스/세럼/앰플,1,2,[NEW/리뉴얼출시] 아이소이 브라이트닝 잡티로즈세럼20ml 더블 한정기획 (+아이...,https://www.oliveyoung.co.kr/store/goods/getGo...,"54,000"
2,에센스/세럼/앰플,1,3,[3월 올영픽]토리든 다이브인 저분자 히알루론산 세럼 50ml 리필 한정 기획 (+...,https://www.oliveyoung.co.kr/store/goods/getGo...,"36,000"
3,에센스/세럼/앰플,1,4,셀리맥스 브라이트닝 모공잡티 세럼 30ml 기획(+10ml),https://www.oliveyoung.co.kr/store/goods/getGo...,"32,000"
4,에센스/세럼/앰플,1,5,[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70m...,https://www.oliveyoung.co.kr/store/goods/getGo...,"50,000"


In [ ]:
from google.colab import files

# 저장한 파일 다운로드
files.download("제품명-에센스_세럼_앰플.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 크림

In [ ]:
# 설치 (최초 1회만)
!pip install -q playwright nest_asyncio
!playwright install chromium

import nest_asyncio
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

nest_asyncio.apply()

# HTML 요청
async def crawl_cream_list_html(playwright_page, page):
    url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010015&pageIdx={page}&rowsPerPage=24"
    await playwright_page.goto(url)
    await playwright_page.wait_for_timeout(3000)
    html = await playwright_page.content()
    return html

# HTML 파싱
def parse_product_list(html, category_name, page_number):
    soup = BeautifulSoup(html, 'lxml')
    items = soup.select(".prd_info")
    data = []

    for idx, item in enumerate(items, start=1):
        try:
            # 제품명
            name_tag = item.select_one(".tx_name")
            product_name = name_tag.get_text(strip=True)

            # 링크
            link = item.select_one("a")["href"]
            full_link = link if link.startswith("http") else "https://www.oliveyoung.co.kr" + link

            # 정가 (없으면 None 처리)
            price_tag = item.select_one(".tx_org .tx_num")
            if not price_tag:
                price_tag = item.select_one(".tx_cur .tx_num")
            price = price_tag.get_text(strip=True) if price_tag else None

            data.append({
                "category": category_name,
                "page": page_number,
                "index": idx,
                "product_name": product_name,
                "link": full_link,
                "price": price
            })
        except Exception as e:
            print(f"⚠️ {idx}번 항목 파싱 실패 (page {page_number}): {e}")
            continue
    return data

# 크롤링 실행
async def main():
    category_name = "크림"
    num_pages = 29
    total_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for i in range(1, num_pages + 1):
            print(f"📦 크롤링 중: {i}페이지")
            html = await crawl_cream_list_html(page, i)
            data = parse_product_list(html, category_name, i)
            total_data.extend(data)

        await browser.close()

    # 저장
    global df, csv_path
    df = pd.DataFrame(total_data)
    csv_path = "제품명-크림.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')

    return df

# 실행
df = await main()
df.head()


📦 크롤링 중: 1페이지
📦 크롤링 중: 2페이지
📦 크롤링 중: 3페이지
📦 크롤링 중: 4페이지
📦 크롤링 중: 5페이지
📦 크롤링 중: 6페이지
📦 크롤링 중: 7페이지
📦 크롤링 중: 8페이지
📦 크롤링 중: 9페이지
📦 크롤링 중: 10페이지
📦 크롤링 중: 11페이지
📦 크롤링 중: 12페이지
📦 크롤링 중: 13페이지
📦 크롤링 중: 14페이지
📦 크롤링 중: 15페이지
📦 크롤링 중: 16페이지
📦 크롤링 중: 17페이지
📦 크롤링 중: 18페이지
📦 크롤링 중: 19페이지
📦 크롤링 중: 20페이지
📦 크롤링 중: 21페이지
📦 크롤링 중: 22페이지
📦 크롤링 중: 23페이지
📦 크롤링 중: 24페이지
📦 크롤링 중: 25페이지
📦 크롤링 중: 26페이지
📦 크롤링 중: 27페이지
📦 크롤링 중: 28페이지
📦 크롤링 중: 29페이지


,category,page,index,product_name,link,price
0,크림,1,1,바이오더마 시카비오 포마드 100ml(리페어 리치 밤),https://www.oliveyoung.co.kr/store/goods/getGo...,"38,000"
1,크림,1,2,[한정수량1+1] 닥터자르트 시카페어 인텐시브 수딩 리페어 크림 50ml 1+1 기획,https://www.oliveyoung.co.kr/store/goods/getGo...,"50,000"
2,크림,1,3,[3월 올영픽]라로슈포제 시카플라스트 밤 B5+ 100ml 기획 (+3ml+1.5m...,https://www.oliveyoung.co.kr/store/goods/getGo...,"40,000"
3,크림,1,4,[3월 올영픽/1+1]에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블기획(+카밍패...,https://www.oliveyoung.co.kr/store/goods/getGo...,"43,000"
4,크림,1,5,에스트라 아토베리어365 크림 80ml 기획 (+하이드로 에센스25ml+세라-히알 ...,https://www.oliveyoung.co.kr/store/goods/getGo...,"33,000"


In [ ]:
from google.colab import files

# 다운로드 실행
files.download("제품명-크림.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 로션

In [ ]:
# 설치 (최초 1회만 실행)
!pip install -q playwright nest_asyncio
!playwright install chromium

import nest_asyncio
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

nest_asyncio.apply()

# HTML 요청
async def crawl_lotion_lotion_list_html(playwright_page, page):
    url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100160001&pageIdx={page}&rowsPerPage=24"
    await playwright_page.goto(url)
    await playwright_page.wait_for_timeout(3000)
    html = await playwright_page.content()
    return html

# HTML 파싱
def parse_product_list(html, category_name, page_number):
    soup = BeautifulSoup(html, 'lxml')
    items = soup.select(".prd_info")
    data = []

    for idx, item in enumerate(items, start=1):
        try:
            # 제품명
            name_tag = item.select_one(".tx_name")
            if not name_tag:
                continue
            product_name = name_tag.get_text(strip=True)

            # 링크
            link_tag = item.select_one("a")
            if not link_tag or not link_tag.get("href"):
                continue
            link = link_tag["href"]
            full_link = link if link.startswith("http") else "https://www.oliveyoung.co.kr" + link

            # 가격: 정가 없으면 현재가로
            price_tag = item.select_one(".tx_org .tx_num")
            if not price_tag:
                price_tag = item.select_one(".tx_cur .tx_num")
            price = price_tag.get_text(strip=True) if price_tag else None

            data.append({
                "category": category_name,
                "page": page_number,
                "index": idx,
                "product_name": product_name,
                "link": full_link,
                "price": price
            })
        except Exception as e:
            print(f"⚠️ {idx}번 항목 파싱 실패 (page {page_number}): {e}")
            continue
    return data

# 크롤링 실행
async def main():
    category_name = "로션 > 로션"
    num_pages = 6  # 24개 × 6 = 144개 수집
    total_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for i in range(1, num_pages + 1):
            print(f"📦 크롤링 중: {i}페이지")
            html = await crawl_lotion_lotion_list_html(page, i)
            data = parse_product_list(html, category_name, i)
            total_data.extend(data)

        await browser.close()

    # 저장
    global df, csv_path
    df = pd.DataFrame(total_data)
    csv_path = "제품명-로션_로션.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    return df

# 실행
df = await main()
df.head()


📦 크롤링 중: 1페이지
📦 크롤링 중: 2페이지
📦 크롤링 중: 3페이지
📦 크롤링 중: 4페이지
📦 크롤링 중: 5페이지
📦 크롤링 중: 6페이지


,category,page,index,product_name,link,price
0,로션 > 로션,1,1,[장벽보습]에스트라 아토베리어365 로션 대용량 300ml 기획 (+30ml+세라-...,https://www.oliveyoung.co.kr/store/goods/getGo...,"59,400"
1,로션 > 로션,1,2,[1등로션]온그리디언츠 스킨 베리어 카밍 로션 기획(220ml+80ml),https://www.oliveyoung.co.kr/store/goods/getGo...,"39,900"
2,로션 > 로션,1,3,에스트라 아토베리어365 로션 150ml 기획 (+에센스 25ml + 리제덤 세럼 ...,https://www.oliveyoung.co.kr/store/goods/getGo...,"33,000"
3,로션 > 로션,1,4,[단독기획] 제로이드 인텐시브 로션 160ml 기획 (+크림 25ml),https://www.oliveyoung.co.kr/store/goods/getGo...,"32,000"
4,로션 > 로션,1,5,비오템 라이프 플랑크톤 피부강화 에센스 인 로션 75ml,https://www.oliveyoung.co.kr/store/goods/getGo...,"98,000"


In [ ]:
from google.colab import files

# 다운로드 실행
files.download("제품명-로션_로션.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 미스트/오일

In [ ]:
# 설치 (최초 1회만)
!pip install -q playwright nest_asyncio
!playwright install chromium

import nest_asyncio
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

nest_asyncio.apply()

# HTML 요청
async def crawl_mist_oil_list_html(playwright_page, page):
    url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010010&pageIdx={page}&rowsPerPage=24"
    await playwright_page.goto(url)
    await playwright_page.wait_for_timeout(3000)
    html = await playwright_page.content()
    return html

# HTML 파싱
def parse_product_list(html, category_name, page_number):
    soup = BeautifulSoup(html, 'lxml')
    items = soup.select(".prd_info")
    data = []

    for idx, item in enumerate(items, start=1):
        try:
            # 제품명
            name_tag = item.select_one(".tx_name")
            product_name = name_tag.get_text(strip=True)

            # 링크 처리
            link = item.select_one("a")["href"]
            full_link = link if link.startswith("http") else "https://www.oliveyoung.co.kr" + link

            # 정가 추출
            price_tag = item.select_one(".tx_org .tx_num")
            if not price_tag:
                price_tag = item.select_one(".tx_cur .tx_num")
            price = price_tag.get_text(strip=True) if price_tag else None

            data.append({
                "category": category_name,
                "page": page_number,
                "index": idx,
                "product_name": product_name,
                "link": full_link,
                "price": price
            })
        except Exception as e:
            print(f"⚠️ {idx}번 항목 파싱 실패 (page {page_number}): {e}")
            continue
    return data

# 크롤링 실행
async def main():
    category_name = "미스트/오일"
    num_pages = 4
    total_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for i in range(1, num_pages + 1):
            print(f"📦 크롤링 중: {i}페이지")
            html = await crawl_mist_oil_list_html(page, i)
            data = parse_product_list(html, category_name, i)
            total_data.extend(data)

        await browser.close()

    # 저장
    global df, csv_path
    df = pd.DataFrame(total_data)
    csv_path = "제품명-미스트_오일.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')

    return df

# 실행
df = await main()
df.head()


📦 크롤링 중: 1페이지
📦 크롤링 중: 2페이지
📦 크롤링 중: 3페이지
📦 크롤링 중: 4페이지


,category,page,index,product_name,link,price
0,미스트/오일,1,1,[NO.1미스트 세럼] 달바 퍼스트 스프레이 세럼 100ml + 100ml 기획,https://www.oliveyoung.co.kr/store/goods/getGo...,"59,800"
1,미스트/오일,1,2,[3월 올영픽/속광탄력] 에스네이처 아쿠아 스쿠알란 미스트 100ml,https://www.oliveyoung.co.kr/store/goods/getGo...,"27,000"
2,미스트/오일,1,3,[재구매1위/퀵보습진정] 바이오힐보 판테셀 리페어시카 크림미스트 120mL+120m...,https://www.oliveyoung.co.kr/store/goods/getGo...,"25,200"
3,미스트/오일,1,4,[1+1]차앤박 프로폴리스 앰플 미스트 250ml 더블기획 (250ml+250ml),https://www.oliveyoung.co.kr/store/goods/getGo...,"27,000"
4,미스트/오일,1,5,[안개분사미스트]에스트라 아토베리어365 크림미스트 120ml,https://www.oliveyoung.co.kr/store/goods/getGo...,"23,000"


In [ ]:
from google.colab import files

# 다운로드 실행
files.download("제품명-미스트_오일.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 패드

In [ ]:
# 설치 (최초 1회만 실행)
!pip install -q playwright nest_asyncio
!playwright install chromium

import nest_asyncio
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

nest_asyncio.apply()

# HTML 요청
async def crawl_pad_list_html(playwright_page, page):
    url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100090004&pageIdx={page}&rowsPerPage=24"
    await playwright_page.goto(url)
    await playwright_page.wait_for_timeout(3000)
    html = await playwright_page.content()
    return html

# HTML 파싱
def parse_product_list(html, category_name, page_number):
    soup = BeautifulSoup(html, 'lxml')
    items = soup.select(".prd_info")
    data = []

    for idx, item in enumerate(items, start=1):
        try:
            # 제품명
            name_tag = item.select_one(".tx_name")
            product_name = name_tag.get_text(strip=True)

            # 링크 처리 (중복 방지)
            link = item.select_one("a")["href"]
            full_link = link if link.startswith("http") else "https://www.oliveyoung.co.kr" + link

            # 정가 (없으면 None)
            price_tag = item.select_one(".tx_org .tx_num")
            if not price_tag:
                price_tag = item.select_one(".tx_cur .tx_num")
            price = price_tag.get_text(strip=True) if price_tag else None

            data.append({
                "category": category_name,
                "page": page_number,
                "index": idx,
                "product_name": product_name,
                "link": full_link,
                "price": price
            })
        except Exception as e:
            print(f"⚠️ {idx}번 항목 파싱 실패 (page {page_number}): {e}")
            continue
    return data

# 크롤링 실행
async def main():
    category_name = "패드"
    num_pages = 11
    total_data = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        for i in range(1, num_pages + 1):
            print(f"📦 크롤링 중: {i}페이지")
            html = await crawl_pad_list_html(page, i)
            data = parse_product_list(html, category_name, i)
            total_data.extend(data)

        await browser.close()

    # 저장
    global df, csv_path
    df = pd.DataFrame(total_data)
    csv_path = "제품명-패드.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')

    return df

# 실행
df = await main()
df.head()


📦 크롤링 중: 1페이지
📦 크롤링 중: 2페이지
📦 크롤링 중: 3페이지
📦 크롤링 중: 4페이지
📦 크롤링 중: 5페이지
📦 크롤링 중: 6페이지
📦 크롤링 중: 7페이지
📦 크롤링 중: 8페이지
📦 크롤링 중: 9페이지
📦 크롤링 중: 10페이지
📦 크롤링 중: 11페이지


,category,page,index,product_name,link,price
0,패드,1,1,[3월 올영픽/파우치 증정] 메디힐 마데카소사이드 흔적 패드 100+100매 토이스...,https://www.oliveyoung.co.kr/store/goods/getGo...,"39,900"
1,패드,1,2,[53관왕/200매] 메디힐 네모패드 100+100매 더블기획 6종 골라담기,https://www.oliveyoung.co.kr/store/goods/getGo...,"39,900"
2,패드,1,3,[3월 올영픽/캐롯시트 증정] 스킨푸드 캐롯 카로틴 카밍 워터 패드 60매 (+10...,https://www.oliveyoung.co.kr/store/goods/getGo...,"26,000"
3,패드,1,4,[단독기획] 바이오던스 겔 토너 패드 60매 기획(+4매 증정) (콜라겐/세라놀),https://www.oliveyoung.co.kr/store/goods/getGo...,"26,000"
4,패드,1,5,[대용량/모공각질] 메디큐브 제로모공패드 2.0 140매 기획 (70매 + 70매 리필),https://www.oliveyoung.co.kr/store/goods/getGo...,"33,000"


In [ ]:
from google.colab import files

# 다운로드 실행
files.download("제품명-패드.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>